In [129]:
import pandas as pd
from pathlib import Path
from collections import Counter
import os
import difflib
import math

In [130]:
DATA_PATH = Path("/home/pengfei/data/pdtb_v2/data/raw/wsj/")

In [131]:
pdtb2 = pd.read_csv("../pdtb2/pdtb2.csv")

/home/pengfei/miniconda3/envs/stanfordnlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


in terms of segment number:
single, double, trible = (38584, 1948, 51)

how many distance away from double span:


    Counter(a).most_common()
    
    
    all of them is within 5

How many distances away:
   
   
   Counter({0: 37713,
         1: 2598,
         2: 193,
         6: 7,
         12: 1,
         9: 3,
         3: 50,
         5: 5,
         18: 2,
         4: 18,
         13: 1,
         8: 4,
         15: 1,
         62: 1,
         19: 1,
         10: 1,
         16: 1})

need correction distance:
    [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 18, 19, 62]

In [6]:
# write functions

In [132]:
def _get_files(data_path=DATA_PATH):
    """Returns (list): [('00', 'wsj_0000'), ..., ('24', 'wsj_2400']"""
    sections = os.listdir(data_path)
    ret = []
    for sec in sorted(sections, key=int):
        for filename in os.listdir(data_path/sec):
            ret.append((sec, filename))
    return ret

def get_batch(section, filenumber):
    """
    Args: 
            section(int)
            filenumber(int)
    Returns:
            (list[int]): list of index with all the file with the same format in that batch"""
    return pdtb2.index[(pdtb2['Section'] == section) & (pdtb2.FileNumber == filenumber)].tolist()


def _get_arg(char_span_list, rawtext):
    arg = ''
    for span in char_span_list:
        arg += rawtext[int(span[0]):int(span[1])]
    return arg

def _get_span_list(span):
    """
    Args:
            span(str): "34..96;97..101"
    Returns:
            ret(list[list[str]]):
            if one arg has one segment: [[34,96]]
            if one arg has two segment: example [[34, 96], [97, 101]]
    """
    if span == '':
        return []
    spans = span.split(';')
    return [[int(k) for k in o.split('..')] for o in spans ]

def check_number_of_errors():
    count = 0
    single = 0
    double = 0
    triple = 0
    files =  _get_files()
    for (section, filename) in files:
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg1
            arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
            arg1_char_span_list = _get_span_list(arg1_char_span_string)
            arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
            if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
                if len(arg1_char_span_list) == 1:
                    single += 1
                elif len(arg1_char_span_list) == 2:
                    double += 1
                else:
                    triple += 1

    print("error numbers for arg1  single:", single, " double: ", double, " triple: ", triple)
    count = 0
    single = 0
    double = 0
    triple = 0
    files =  _get_files()
    for (section, filename) in files:
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg2
            arg2_char_span_string = pdtb2.loc[i, 'Arg2_SpanList']
            arg2_char_span_list = _get_span_list(arg2_char_span_string)
            arg2_rawtext = pdtb2.loc[i, 'Arg2_RawText']
            if _get_arg(arg2_char_span_list, rawtext) != arg2_rawtext:
                if len(arg2_char_span_list) == 1:
                    single += 1
                elif len(arg2_char_span_list) == 2:
                    double += 1
                else:
                    triple += 1
    print("error numbers for arg2  single:", single, " double: ", double, " triple: ", triple)
    
def get_length(list):
    length = 0
    for o in list:
        length += o[1] - o[0]
    return length


def get_start_end(span_list, rawtext, tolerance_size):
    if span_list == []:
        return 0, len(rawtext)-1
    tolerance_size += 1
    start = span_list[0][0]
    end = span_list[len(span_list)-1][1]
    if start-tolerance_size<0:
        start = 0
    else:
        start -= tolerance_size
    if end + tolerance_size>len(rawtext)-1:
        end = len(rawtext) -1
    else:
        end += len(rawtext)+tolerance_size
    
    return start, end

def correct_span(span_list, arg_rawtext, rawtext, tolerance_size=None):
    """
    Args: 
            span_list[list[[int, int], ...]]
            arg_rawtext(str)
            rawtext(str): doc raw text
    Returns: 
            corrected_span(str): [34, 39]
    """
    corrected_span = ''
    if tolerance_size == None:
        tolerance_size = abs(get_length(span_list) - len(arg_rawtext))
    print(tolerance_size)
    start, end = get_start_end(span_list, rawtext, tolerance_size)

    s = difflib.SequenceMatcher(lambda x: x=="\n", rawtext[start: end], arg_rawtext)
    for i, j, n in s.get_matching_blocks():
        if n!=0:
            corrected_span += f'{start+i}..{start+i+n};' # here is a offset of 2 char we need to consider
    corrected_span = corrected_span[:-1]
    return corrected_span

def sanity_check(corrected_span, rawtext, arg_rawtext):
    """Args: 
            corrected_span(str)
            rawtext(str): doc string
            arg_rawtext(str): gold argument string
    Returns: 
            (boolean): the corrected span is corrected"""
    span_list_corrected = _get_span_list(corrected_span)
    corrected_arg = _get_arg(span_list_corrected, rawtext)
    return corrected_arg == arg_rawtext

In [46]:
check_number_of_errors()

error numbers for arg1  single: 36317  double:  2309  triple:  436
error numbers for arg2  single: 35507  double:  3494  triple:  61


In [11]:
def correct_docs():
    files =  _get_files()
    count = 0
    print(" --- correction start ----")
    for (section, filename) in files:
        print('section:', section, ' files: ', filename)
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg1
            arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
            arg1_char_span_list = _get_span_list(arg1_char_span_string)
            arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
            if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
                corrected_span = correct_span(arg1_char_span_list, arg1_rawtext, rawtext)
                count += 1
                check = sanity_check(corrected_span, rawtext, arg1_rawtext)
                if check == False:
#                     error_range = 1
#                     while sanity_check(corrected_span, rawtext, arg1_rawtext) == False and error_range < 100:
#                         corrected_span = correct_span(arg1_char_span_list, arg1_rawtext, rawtext, error_range)
#                         error_range += 1
#                     if sanity_check(corrected_span, rawtext, arg1_rawtext) == False:
#                         print("Attention!")
                    print("Attention!")
                pdtb2.loc[i, 'Arg1_SpanList'] = corrected_span
                

#             # for arg2
#             arg2_char_span_string = pdtb2.loc[i, 'Arg2_SpanList']
#             arg2_char_span_list = _get_span_list(arg2_char_span_string)
#             arg2_rawtext = pdtb2.loc[i, 'Arg2_RawText']
#             if _get_arg(arg2_char_span_list, rawtext) != arg2_rawtext:
#                 corrected_span = correct_span(arg2_char_span_list, arg2_rawtext, rawtext)
#                 check = sanity_check(corrected_span, rawtext, arg1_rawtext)
#                 count += 1
#                 if check == False:
#                     print("Attention!")
#                 pdtb2.loc[i, 'Arg2_SpanList'] = corrected_span
    print("--- correction finished ------")
    return pdtb2, count

In [12]:
pdtb2, count = correct_docs()

 --- correction start ----
section: 00  files:  wsj_0076
section: 00  files:  wsj_0063
Attention!
section: 00  files:  wsj_0069
section: 00  files:  wsj_0033
section: 00  files:  wsj_0095
section: 00  files:  wsj_0038
section: 00  files:  wsj_0051
section: 00  files:  wsj_0021
section: 00  files:  wsj_0015
section: 00  files:  wsj_0066
section: 00  files:  wsj_0094
section: 00  files:  wsj_0025
section: 00  files:  wsj_0093
section: 00  files:  wsj_0010
section: 00  files:  wsj_0023
section: 00  files:  wsj_0059
section: 00  files:  wsj_0027
section: 00  files:  wsj_0028
section: 00  files:  wsj_0065
section: 00  files:  wsj_0099
section: 00  files:  wsj_0012
section: 00  files:  wsj_0035
section: 00  files:  wsj_0039
Attention!
section: 00  files:  wsj_0058
section: 00  files:  wsj_0074
section: 00  files:  wsj_0004
section: 00  files:  wsj_0080
section: 00  files:  wsj_0070
section: 00  files:  wsj_0037
section: 00  files:  wsj_0008
section: 00  files:  wsj_0056
section: 00  files:  

section: 02  files:  wsj_0272
section: 02  files:  wsj_0216
section: 02  files:  wsj_0299
section: 02  files:  wsj_0215
section: 02  files:  wsj_0249
section: 02  files:  wsj_0204
section: 02  files:  wsj_0233
section: 02  files:  wsj_0256
section: 02  files:  wsj_0288
section: 02  files:  wsj_0295
section: 02  files:  wsj_0231
section: 02  files:  wsj_0214
section: 02  files:  wsj_0250
section: 02  files:  wsj_0200
section: 02  files:  wsj_0271
section: 02  files:  wsj_0241
section: 02  files:  wsj_0212
section: 02  files:  wsj_0267
section: 02  files:  wsj_0238
section: 02  files:  wsj_0210
section: 02  files:  wsj_0283
section: 02  files:  wsj_0286
section: 02  files:  wsj_0280
section: 03  files:  wsj_0341
section: 03  files:  wsj_0335
section: 03  files:  wsj_0382
section: 03  files:  wsj_0308
section: 03  files:  wsj_0306
section: 03  files:  wsj_0380
section: 03  files:  wsj_0378
section: 03  files:  wsj_0364
section: 03  files:  wsj_0317
section: 03  files:  wsj_0325
section: 0

section: 05  files:  wsj_0564
Attention!
section: 05  files:  wsj_0530
Attention!
Attention!
section: 05  files:  wsj_0558
section: 05  files:  wsj_0506
section: 05  files:  wsj_0525
section: 05  files:  wsj_0582
section: 05  files:  wsj_0507
section: 05  files:  wsj_0595
section: 05  files:  wsj_0527
section: 05  files:  wsj_0511
section: 05  files:  wsj_0577
section: 05  files:  wsj_0578
section: 05  files:  wsj_0586
section: 05  files:  wsj_0542
section: 05  files:  wsj_0522
section: 05  files:  wsj_0589
section: 05  files:  wsj_0521
section: 05  files:  wsj_0556
section: 05  files:  wsj_0559
section: 05  files:  wsj_0532
section: 05  files:  wsj_0555
section: 05  files:  wsj_0546
section: 05  files:  wsj_0524
section: 05  files:  wsj_0554
section: 05  files:  wsj_0535
section: 05  files:  wsj_0543
section: 05  files:  wsj_0560
Attention!
section: 05  files:  wsj_0513
section: 05  files:  wsj_0549
section: 05  files:  wsj_0562
section: 05  files:  wsj_0528
section: 05  files:  wsj_0

section: 08  files:  wsj_0825
section: 08  files:  wsj_0816
section: 08  files:  wsj_0896
section: 08  files:  wsj_0852
section: 08  files:  wsj_0886
section: 08  files:  wsj_0829
section: 08  files:  wsj_0876
section: 08  files:  wsj_0803
section: 08  files:  wsj_0885
section: 08  files:  wsj_0823
section: 08  files:  wsj_0889
section: 08  files:  wsj_0830
section: 08  files:  wsj_0809
section: 08  files:  wsj_0805
section: 08  files:  wsj_0814
section: 08  files:  wsj_0863
section: 08  files:  wsj_0800
section: 08  files:  wsj_0887
section: 08  files:  wsj_0848
section: 08  files:  wsj_0835
section: 08  files:  wsj_0822
section: 08  files:  wsj_0847
section: 08  files:  wsj_0854
section: 08  files:  wsj_0806
section: 08  files:  wsj_0862
section: 08  files:  wsj_0892
section: 08  files:  wsj_0817
section: 08  files:  wsj_0888
section: 08  files:  wsj_0853
section: 08  files:  wsj_0883
section: 08  files:  wsj_0898
section: 08  files:  wsj_0828
section: 08  files:  wsj_0858
section: 0

KeyboardInterrupt: 

In [ ]:
pdtb2.to_csv('pdtb2.csv')

In [93]:
def correct_span(span_list, arg_rawtext, rawtext, tolerance_size=None):
    """
    Args: 
            span_list[list[[int, int], ...]]
            arg_rawtext(str)
            rawtext(str): doc raw text
    Returns: 
            corrected_span(str): [34, 39]
    """
    corrected_span = ''
    if tolerance_size == None:
        tolerance_size = abs(get_length(span_list) - len(arg_rawtext))
    print(tolerance_size, len(arg_rawtext))
    start, end = get_start_end(span_list, rawtext, tolerance_size)

    s = difflib.SequenceMatcher(lambda x: x in punctuation, rawtext[start: end], arg_rawtext)
    for i, j, n in s.get_matching_blocks():
        if n!=0:
            corrected_span += f'{start+i}..{start+i+n};' # here is a offset of 2 char we need to consider
    corrected_span = corrected_span[:-1]
    return corrected_span

In [96]:
## testing code, do not delete
files =  _get_files()
count = 0
for (section, filename) in files[:103]:
    with open(DATA_PATH/section/filename, encoding='latin-1') as f:
        rawtext = f.read()
    
    
    section = int(section)
    filenumber = int(filename[-2:])
    batch = get_batch(section, filenumber)
    for i in batch:
        arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
        arg1_char_span_list = _get_span_list(arg1_char_span_string)
        arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
        if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
            corrected_span = correct_span(arg1_char_span_list, arg1_rawtext, rawtext)
            span_list_corrected = _get_span_list(corrected_span)
            corrected_arg = _get_arg(span_list_corrected, rawtext)
#             print(arg1_rawtext== corrected_arg)
            k = 1/0

28 205


ZeroDivisionError: division by zero

In [97]:
rawtext, arg1_rawtext

('.START \n\nSea Containers Ltd. said it might increase the price of its $70-a-share buy-back plan if pressed by Temple Holdings Ltd., which made an earlier tender offer for Sea Containers. \n\nSea Containers, a Hamilton, Bermuda-based shipping concern, said Tuesday that it would sell $1.1 billion of assets and use some of the proceeds to buy about 50% of its common shares for $70 apiece. \n\nThe move is designed to ward off a hostile takeover attempt by two European shipping concerns, Stena Holding AG and Tiphook PLC.\nIn May, the two companies, through their jointly owned holding company, Temple, offered $50 a share, or $777 million, for Sea Containers.\nIn August, Temple sweetened the offer to $63 a share, or $963 million. \n\nYesterday, Sea Containers\' chief executive officer, James Sherwood, said in an interview that, under the asset-sale plan, Sea Containers would end up with a cash surplus of approximately $620 million.\nAbout $490 million of that would be allocated to the buy-

In [127]:
string = ''
start, end = get_start_end(arg1_char_span_list, rawtext, 20)

s = difflib.SequenceMatcher(None, rawtext[start:end], arg1_rawtext.# lambda x: x in punctuation
for i, j, n in s.get_matching_blocks():
    if n!=0:
        print(f'{rawtext[start+i: start+i+n]}')
        string += f'{rawtext[start+i: start+i+n]}'

y, characterizing it as a "highly conditional device designed to entrench management, confuse shareholders and prevent them from accepting our superior cash offer." 

A spokesman for Temple estimated that Sea Containers' plan -- if all the asset sales materialize -- would result in shareholders receiving only $36 to $45 a share in cash.
The lower figures, the spokesman said, would stem from preferred shares being converted to common stock and the possibility that Sea Containers' subsidiaries might be required to place their shares in the open market. 

Temple added that Sea Containers is still mired in legal problems in Bermuda, where the Supreme Court has temporarily barred Sea Containers from buying back its own stock in a case brought by Stena and Tiphook. {The court has indicated it will rule on the case by the end of the month.} 

Temple also said Sea Containers' plan raises "numerous legal, regulatory, financial and fairness issues," but didn't elaborate. 

Mr. Sherwood said reac

In [136]:
string = ''
start, end = get_start_end(arg1_char_span_list, rawtext, 20)

s = difflib.SequenceMatcher(None, strip(rawtext), strip(arg1_rawtext))# lambda x: x in punctuation
for i, j, n in s.get_matching_blocks():
    if n!=0:
        print(f'{strip(rawtext)[i: i+n]}')
        string += f'{strip(rawtext)[i: i+n]}'

asahighlyconditionaldevicedesignedtoentrenchmanagementconfuseshareholdersandpreventthemfromacceptingoursuperiorcashoffer
thatSeaContainersisstillmiredinlegalproblemsinBermuda


In [126]:
for i in s.get_grouped_opcodes():
    print(i)

[('delete', 0, 3, 0, 0), ('equal', 3, 6, 0, 3)]
[('equal', 19, 22, 16, 19), ('replace', 22, 88, 19, 20), ('equal', 88, 91, 20, 23)]
[('equal', 95, 98, 27, 30), ('replace', 98, 184, 30, 31)]


In [135]:
def strip(string):
    punctuation = """!"#&'*+,-..../:;<=>?@[\]^_`|~""" + "``" + "''" + '\n '
    return ''.join(ch for ch in string if ch not in punctuation)

In [137]:
strip(arg1_rawtext)

'asahighlyconditionaldevicedesignedtoentrenchmanagementconfuseshareholdersandpreventthemfromacceptingoursuperiorcashofferthatSeaContainersisstillmiredinlegalproblemsinBermuda'

In [138]:
string

'asahighlyconditionaldevicedesignedtoentrenchmanagementconfuseshareholdersandpreventthemfromacceptingoursuperiorcashofferthatSeaContainersisstillmiredinlegalproblemsinBermuda'

In [140]:
strip(arg1_rawtext) == string

True